<a href="https://colab.research.google.com/github/javiercarro/case-aq/blob/develop/src/U_tad_AQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicio

In [1]:
# Montaje de Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Datos clima
df_model = pd.read_csv('./drive/My Drive/U-tad/UseCaseAQ/data/Ladreda2016.csv', sep=',', encoding='ISO-8859-1')
df_test = pd.read_csv('./drive/My Drive/U-tad/UseCaseAQ/data/Ladreda2017.csv', sep=',', encoding='ISO-8859-1')

#Datos

## Datos de contaminación (Ayto. de Madrid)
Descarga de datos para ver el formato complejo.

## Datos meteorológicos (AEMET)
Descarga de datos (ver que no hay histórico de datos horarios)


> **Actividad**:
> * Comprobar que no hay histórico de datos horarios
> * Descargar histórico de datos de AEMET de un año







## Preparación de datasets de trabajo

In [0]:
df_model.describe()

### Correlaciones
 * Utilizando datos de 2016 y comprobando con 2017
 * Utilizando método "forward selection"

In [0]:
matplotlib.rcParams['figure.figsize'] = (20.0, 8.0)

x_values = [datetime.strptime(d,"%Y-%m-%d").date() for d in df_model['Fecha']]
y1_values = df_model['NO2_medio']
y2_values = df_model['NO2_mediano']
y3_values = df_model['NO2_maximo']
ax = plt.gca()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
locator = mdates.DayLocator(bymonthday=1)
ax.xaxis.set_major_locator(locator)
plt.plot(x_values, y1_values, color='green', label='NO2 Medio')
plt.plot(x_values, y2_values, color='blue', label='NO2 Mediano')
plt.fill_between(x_values, y3_values, facecolor='blue', alpha=0.4, edgecolor='black', label='NO2 Máximo')
plt.legend(loc="upper right")
print(np.std(df_model['NO2_medio']))
print(np.std(df_model['NO2_mediano']))
print(np.std(df_model['NO2_maximo']))

In [0]:
var_corr = 'NO2_medio'
matplotlib.rcParams['figure.figsize'] = (10.0, 15.0)
df_model.corr()[var_corr].sort_values(ascending=False).plot(kind='barh')
plt.xlim(-1, 1)
plt.title("Correlation for variable" + var_corr)

In [0]:
var_corr = 'NO2_mediano'
matplotlib.rcParams['figure.figsize'] = (10.0, 15.0)
df_model.corr()[var_corr].sort_values(ascending=False).plot(kind='barh')
plt.xlim(-1, 1)
plt.title("Correlation for variable" + var_corr)

In [0]:
var_corr = 'NO2_maximo'
matplotlib.rcParams['figure.figsize'] = (10.0, 15.0)
df_model.corr()[var_corr].sort_values(ascending=False).plot(kind='barh')
plt.xlim(-1, 1)
plt.title("Correlation for variable" + var_corr)

In [0]:
# Correlaciones
modelCorr = np.triu(df_model.corr())
sns.heatmap(df_model.corr(), vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', mask=modelCorr,  square=False)

In [0]:
testCorr = np.triu(df_test.corr())
sns.heatmap(df_test.corr(), vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', mask=testCorr,  square=False)

In [0]:
for i,var in enumerate(df_model.corr().columns):
    mv = df_model.corr().iloc[i+1:][var]
    mv_related = mv[(mv > 0.9) | (mv < -0.9)]
    if len(mv_related) > 0:
        print(var+":")
        print("--------------------------")
        print(mv_related)
        print("")

In [0]:
from sklearn.datasets import load_breast_cancer
data  = load_breast_cancer(return_X_y=False)

import pandas as pd

df=pd.DataFrame(data.data[:,:5])
df.columns = data.feature_names[:5]



import seaborn as sns;
import matplotlib.pyplot as plt
g= sns.pairplot(df, diag_kind= 'hist', 
             vars=df.columns[:-1],
             plot_kws=dict(alpha=0.5), 
             diag_kws=dict(alpha=0.5))
plt.show()

### Limpieza

In [0]:
# Eliminamos algunas columnas: las de NO2 del mismo dia, ya que no las tendremos en la prediccion
# tambien otras columnas irrelevantes o que introducen ruido

# Estaciones y NO2 del mismo dia
df_model.drop(['Estacion_clima', 'Estacion_contaminacion', 'NO2_mediano', 'NO2_maximo', 'NO2_minimo'], axis=1, inplace=True)

# Horas donde se producen valores maximos, minimos
df_model.drop(['HoraTmin', 'HoraTmax', 'HoraRacha', 'HoraPresMax', 'HoraPresMin', 'HoraTmin_dia_anterior', 'HoraTmax_dia_anterior', 'HoraRacha_dia_anterior', 'HoraPresMax_dia_anterior', 'HoraPresMin_dia_anterior'], axis=1, inplace=True)

# temperaturas
# df_model.drop(['Tmed', 'Tmin', 'Tmax'], axis=1, inplace=True)

# Viento
# df_model.drop(['DirViento', 'VelViento', 'Racha'], axis=1, inplace=True)

# presion
# df_model.drop(['PresMax', 'PresMin'], axis=1, inplace=True)

In [0]:
# Estaciones y NO2 del mismo dia
df_test.drop(['Estacion_clima', 'Estacion_contaminacion', 'NO2_mediano', 'NO2_maximo', 'NO2_minimo'], axis=1, inplace=True)

# Horas donde se producen valores maximos, minimos
df_test.drop(['HoraTmin', 'HoraTmax', 'HoraRacha', 'HoraPresMax', 'HoraPresMin', 'HoraTmin_dia_anterior', 'HoraTmax_dia_anterior', 'HoraRacha_dia_anterior', 'HoraPresMax_dia_anterior', 'HoraPresMin_dia_anterior'], axis=1, inplace=True)

# temperaturas
# df_test.drop(['Tmed', 'Tmin', 'Tmax'], axis=1, inplace=True)

# Viento
# df_test.drop(['DirViento', 'VelViento', 'Racha'], axis=1, inplace=True)

# presion
# df_test.drop(['PresMax', 'PresMin'], axis=1, inplace=True)

In [0]:
# Guardamos las fechas para obtener un dataframe resultado de la forma: fecha/valor real/prediccion

df_fecha = pd.DataFrame()
df_fecha = df_test[['Fecha']].copy()

In [0]:
# Obtenemos los dummies

df_model = pd.get_dummies(df_model, columns=['Estaciones_Anio', 'Mes', 'Dia_Semana', 'Fin_Semana', 'Festivo', 'Laborable', 'Pre_festivo', 'Escolar', 'Festivo_escolar', 'LLuvia', 'NivelViento'])

In [0]:
df_test = pd.get_dummies(df_test, columns=['Estaciones_Anio', 'Mes', 'Dia_Semana', 'Fin_Semana', 'Festivo', 'Laborable', 'Pre_festivo', 'Escolar', 'Festivo_escolar', 'LLuvia', 'NivelViento'])

In [0]:
# Eliminamos mas columnas que introducen ruido

df_model.drop(['PresMax', 'PresMax_dia_anterior', 'PresMin', 'PresMin_dia_anterior'], axis=1, inplace=True)
df_test.drop(['PresMax', 'PresMax_dia_anterior', 'PresMin', 'PresMin_dia_anterior'], axis=1, inplace=True)

In [0]:
# Completamos el dataframe test

df_test['Estaciones_Anio_Otono'] = 0
df_test['Mes_julio'] = 0
df_test['Mes_agosto'] = 0
df_test['Mes_septiembre'] = 0
df_test['Mes_octubre'] = 0
df_test['Mes_noviembre'] = 0
df_test['Mes_diciembre'] = 0

In [0]:
# Guardamos la columna target
no2_model = df_model['NO2_medio']
no2_test = df_test['NO2_medio']

### Preparación de datasets para algoritmos de ML
Tomamos los datos de 2016 como datos de entrenamiento, y los de 2017 como datos de test. En este caso, esta proporción 50/50, frente a otras proporciones que se suelen utilizar, es aconsejable debido al comportamiento estacional (no sólo semanal, sino sobre todo anual a causa de las estaciones del año) aconseja esta división.

# Predicciones

In [0]:
# Ridge
def pred_ridge(X_train, y_train, X_test, y_test, name):
    ridgeret = linear_model.Ridge() 
    grid = GridSearchCV(ridgeret,parametros,scoring='neg_mean_squared_error')
    grid.fit(X_train,y_train)
    alfa_ridge=grid.best_params_['alpha']

    # # Instantiate model
    ridgeret = linear_model.Ridge(alpha=alfa_ridge) 
    # ridgeret = linear_model.RidgeCV()

    # # Fit Model
    ridgeret.fit(X_train,y_train)

    # # Predict
    y_pred_ridge = ridgeret.predict(X_test)

    results['Ridge_'+(str(name))] = y_pred_ridge

    scores.loc[('Ridge'), ('R2_'+(str(name)))] = "{0:.6f}".format(metrics.r2_score(y_test, y_pred_ridge))
    scores.loc[('Ridge'), ('RMSE_'+(str(name)))] = "{0:.6f}".format(np.sqrt(metrics.mean_squared_error(y_test, y_pred_ridge)))

In [0]:
# RandomForestRegressor
def pred_RF(X_train, y_train, X_test, y_test, name):
    # Instantiate mode
    # RF = RandomForestRegressor()
    RF = RandomForestRegressor(n_estimators=100, criterion='mse')

    # Fit Model
    RF.fit(X_train,y_train)

    # Predict
    y_pred_RF = RF.predict(X_test)

    # print("Features sorted by their score:")
    # print(sorted(zip(map(lambda x: round(x, 4), RF.feature_importances_), X_train.columns), reverse=True))

    results['RF_'+(str(name))] = y_pred_RF
    scores.loc[('RF'), ('R2_'+(str(name)))] = "{0:.6f}".format(metrics.r2_score(y_test, y_pred_RF))
    scores.loc[('RF'), ('RMSE_'+(str(name)))] = "{0:.6f}".format(np.sqrt(metrics.mean_squared_error(y_test, y_pred_RF)))

In [0]:
def prediction(X_train, y_train, X_test, y_test, name):
    pred_ridge(X_train, y_train, X_test, y_test, name)
    pred_RF(X_train, y_train, X_test, y_test, name)

In [0]:
# prediccion con NO2, clima e informacion de la fecha

In [0]:
df_model.drop('Fecha', axis=1, inplace=True)
df_test.drop('Fecha', axis=1, inplace=True)

# Split data
X_train = df_model.drop('NO2_medio', axis=1)
y_train = df_model['NO2_medio'].values

X_test =  df_test.drop('NO2_medio', axis=1)
y_test = df_test['NO2_medio'].values

parametros = { 'alpha': np.arange(0, 50, 1)}

results = pd.DataFrame()
scores = pd.DataFrame(index=['Ridge', 'RF'])
results['Test'] = y_test

prediction(X_train, y_train, X_test, y_test, 'all')

In [0]:
# prediccion con clima e informacion de la fecha

In [0]:
# Sin las variables de contaminacion:
df_model.drop([col for col in df_model.columns if "NO2" in col], axis=1, inplace=True)
df_test.drop([col for col in df_test.columns if "NO2" in col], axis=1, inplace=True)

df_model['NO2_medio'] = no2_model
df_test['NO2_medio'] = no2_test

In [0]:
# Split data
X_train = df_model.drop('NO2_medio', axis=1)
y_train = df_model['NO2_medio'].values

X_test =  df_test.drop('NO2_medio', axis=1)
y_test = df_test['NO2_medio'].values

# prediccion con clima y fecha
prediction(X_train, y_train, X_test, y_test, 'cl_date')

In [0]:
# Sin las variables de la fecha
contlluvia_model = df_model['Contador_sin_lluvia']
contlluvia_test = df_test['Contador_sin_lluvia']
contviento_model = df_model['Contador_sin_viento']
contviento_test = df_test['Contador_sin_viento']

df_model.drop([col for col in df_model.columns if "Contador" in col], axis=1, inplace=True)
df_test.drop([col for col in df_test.columns if "Contador" in col], axis=1, inplace=True)
df_model.drop([col for col in df_model.columns if "Estaciones" in col], axis=1, inplace=True)
df_test.drop([col for col in df_test.columns if "Estaciones" in col], axis=1, inplace=True)
df_model.drop([col for col in df_model.columns if "Mes" in col], axis=1, inplace=True)
df_test.drop([col for col in df_test.columns if "Mes" in col], axis=1, inplace=True)
df_model.drop([col for col in df_model.columns if "Semana" in col], axis=1, inplace=True)
df_test.drop([col for col in df_test.columns if "Semana" in col], axis=1, inplace=True)
df_model.drop([col for col in df_model.columns if "estivo" in col], axis=1, inplace=True)
df_test.drop([col for col in df_test.columns if "estivo" in col], axis=1, inplace=True)
df_model.drop([col for col in df_model.columns if "Laborable" in col], axis=1, inplace=True)
df_test.drop([col for col in df_test.columns if "Laborable" in col], axis=1, inplace=True)

df_model['Contador_sin_lluvia'] = contlluvia_model
df_test['Contador_sin_lluvia'] = contlluvia_test
df_model['Contador_sin_viento'] = contviento_model
df_test['Contador_sin_viento'] = contviento_test

In [0]:
# Split data
X_train = df_model.drop('NO2_medio', axis=1)
y_train = df_model['NO2_medio'].values

X_test =  df_test.drop('NO2_medio', axis=1)
y_test = df_test['NO2_medio'].values

# prediccion con clima
prediction(X_train, y_train, X_test, y_test, 'cl')

In [0]:
# Puntuaciones
scores

In [0]:
# Obtenemos el df resultado
results = pd.merge(df_fecha, results, left_index=True, right_index=True)
results = results.sort_index()
results

In [0]:
matplotlib.rcParams['figure.figsize'] = (20.0, 8.0)

x_values = [datetime.strptime(d,"%Y-%m-%d").date() for d in results['Fecha']]
y1_values = results['Test']
y3_values = results['Ridge_all']
y4_values = results['RF_all']
ax = plt.gca()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
locator = mdates.DayLocator(bymonthday=1)
ax.xaxis.set_major_locator(locator)
plt.plot(x_values, y4_values, color='blue', label='RF_all')
plt.plot(x_values, y3_values, color='green', label='Ridge_all')
plt.fill_between(x_values, y1_values, facecolor='blue', alpha=0.4, edgecolor='black', label='Test')
plt.legend(loc="upper right")
#print(np.std(df_model['Test']))
#print(np.std(df_model['Lasso_all']))
#print(np.std(df_model['Ridge_all']))